In [1]:
import json
import pandas as pd
import spacy
from collections import defaultdict, Counter
from datasets import Dataset, DatasetDict, load_from_disk
from spacy.matcher import Matcher
from spacy.lang.en import English
from spacy.tokens import Span
from spacy import displacy

In [2]:
dictionary = pd.read_csv('dictionary.csv')

In [3]:
dictionary

,nodeid,name
0,100000,ABBYY
1,100000,ABBY
2,100002,Adobe Analytics
3,100003,Adobe Audience Manager
4,100004,Adobe CC
...,...,...
564,100564,ZMQ
565,100565,Zoho
566,100566,Zookeeper
567,100567,Zoom


In [4]:
dictio = defaultdict(dict)
for number in dictionary.index:
    if str(dictionary.at[number, 'nodeid']) in dictio:
        dictio[str(dictionary.at[number, 'nodeid'])].append(dictionary.at[number, 'name'])
    else:
        dictio[str(dictionary.at[number, 'nodeid'])] = [dictionary.at[number, 'name']]

In [5]:
dictio

defaultdict(dict,
            {'100000': ['ABBYY', 'ABBY'],
             '100002': ['Adobe Analytics'],
             '100003': ['Adobe Audience Manager'],
             '100004': ['Adobe CC'],
             '100005': ['Adobe Launch'],
             '100006': ['Adobe Target'],
             '100007': ['Adobe XD', 'AdobeXD'],
             '100009': ['ADONIS'],
             '100010': ['AFT Fathom'],
             '930332': ['Airflow'],
             '100012': ['AKS'],
             '100013': ['Alation'],
             '100014': ['Altium Designer'],
             '100015': ['Amadeus'],
             '100016': ['Amazon Connect'],
             '100017': ['Anaplan'],
             '93055': ['Android'],
             '100019': ['Android Studio'],
             '100020': ['Angular'],
             '930454': ['AngularJS'],
             '930751': ['Ansible'],
             '100023': ['Ansys'],
             '100024': ['Ant'],
             '100025': ['API Gateway'],
             '100026': ['APL'],
             '1

In [6]:
nlp = English()
matcher = Matcher(nlp.vocab)
ruler = nlp.add_pipe('entity_ruler')

In [7]:
temp = []
cnt = Counter()
tech_count = []

In [8]:
def find_2nd(string, substring):
   return string.find(substring, string.find(substring) + 1)

In [9]:
for idnmb in dictio:
    if idnmb.startswith('9300'):
        key = 'Data Mining and Machine Learning'
    elif idnmb.startswith('9301'):
        key = 'Programming Languages'
    elif idnmb.startswith('9302'):
        key = 'Cloud Computing'
    elif idnmb.startswith('9303'):
        key = 'Databases and Data Warehouses'
    elif idnmb.startswith('9304'):
        key = 'Web Development and Data Visualization'
    elif idnmb.startswith('9305'):
        key = 'Operating Systems'
    elif idnmb.startswith('9306'):
        key = 'Virtualization and Containerization'
    elif idnmb.startswith('9307'):
        key = 'Collaboration and Automation'
    elif idnmb.startswith('9308'):
        key = 'Computer Network Tools'
    elif idnmb.startswith('9309'):
        key = 'Software Development Tools'
    elif idnmb.startswith('9310'):
        key = 'Office Software'
    elif idnmb.startswith('1'):
        key = 'Infrequent'
    for entity in dictio[idnmb]:
        if entity.count(" ") == 2:
            space = find_2nd(entity, " ")
            sub_pattern1 = [{"LOWER": entity[:entity.find(" ")].lower()}, {"LOWER": entity[entity.find(" ") + 1: space].lower()}, {"LOWER": entity[space + 1:].lower()}]
            sub_pattern2 = [{"LOWER": entity[:entity.find(" ")].lower()}, {"TEXT": "-"}, {"LOWER": entity[entity.find(" ") + 1: space].lower()}, {"LOWER": entity[space + 1:].lower()}]
            sub_pattern3 = [{"LOWER": entity[:entity.find(" ")].lower()}, {"TEXT": "-"}, {"LOWER": entity[entity.find(" ") + 1: space].lower()}, {"TEXT": "-"}, {"LOWER": entity[space + 1:].lower()}]
            sub_pattern4 = [{"LOWER": entity[:entity.find(" ")].lower()}, {"LOWER": entity[entity.find(" ") + 1: space].lower()}, {"TEXT": "-"}, {"LOWER": entity[space + 1:].lower()}]
            sub_pattern5 = [{"LOWER": "-" + entity[:entity.find(" ")].lower()}, {"LOWER": entity[entity.find(" ") + 1: space].lower()}, {"LOWER": entity[space + 1:].lower()}]
            sub_pattern6 = [{"LOWER": "-" + entity[:entity.find(" ")].lower()}, {"TEXT": "-"}, {"LOWER": entity[entity.find(" ") + 1: space].lower()}, {"LOWER": entity[space + 1:].lower()}]
            sub_pattern7 = [{"LOWER": "-" + entity[:entity.find(" ")].lower()}, {"TEXT": "-"}, {"LOWER": entity[entity.find(" ") + 1: space].lower()}, {"TEXT": "-"}, {"LOWER": entity[space + 1:].lower()}]
            sub_pattern8 = [{"LOWER": "-" + entity[:entity.find(" ")].lower()}, {"LOWER": entity[entity.find(" ") + 1: space].lower()}, {"TEXT": "-"}, {"LOWER": entity[space + 1:].lower()}]
            pattern = [
                sub_pattern1,
                sub_pattern2,
                sub_pattern3,
                sub_pattern4,
                sub_pattern5,
                sub_pattern6,
                sub_pattern7,
                sub_pattern8
            ]
            patterns = [
                {"label": key, "pattern": sub_pattern1, "id": idnmb},
                {"label": key, "pattern": sub_pattern2, "id": idnmb},
                {"label": key, "pattern": sub_pattern3, "id": idnmb},
                {"label": key, "pattern": sub_pattern4, "id": idnmb},
                {"label": key, "pattern": sub_pattern5, "id": idnmb},
                {"label": key, "pattern": sub_pattern6, "id": idnmb},
                {"label": key, "pattern": sub_pattern7, "id": idnmb},
                {"label": key, "pattern": sub_pattern8, "id": idnmb}
            ]
        elif " " in entity and len(entity[entity.find(" ") + 1:]) == 1:
            sub_pattern1 = [{"LOWER": entity[:entity.find(" ")].lower()}, {"LOWER": entity[entity.find(" ") + 1:].lower()}]
            sub_pattern2 = [{"LOWER": entity[:entity.find(" ")].lower()}, {"TEXT": "-"}, {"LOWER": entity[entity.find(" ") + 1:].lower()}]
            sub_pattern3 = [{"LOWER": "-" + entity[:entity.find(" ")].lower()}, {"LOWER": entity[entity.find(" ") + 1:].lower()}]
            sub_pattern4 = [{"LOWER": "-" + entity[:entity.find(" ")].lower()}, {"TEXT": "-"}, {"LOWER": entity[entity.find(" ") + 1:].lower()}]
            sub_pattern5 = [{"LOWER": entity[:entity.find(" ")].lower()}, {"LOWER": entity[entity.find(" ") + 1:].lower() + "."}]
            sub_pattern6 = [{"LOWER": entity[:entity.find(" ")].lower()}, {"TEXT": "-"}, {"LOWER": entity[entity.find(" ") + 1:].lower() + "."}]
            pattern = [
                sub_pattern1,
                sub_pattern2,
                sub_pattern3,
                sub_pattern4,
                sub_pattern5,
                sub_pattern6
            ]
            patterns = [
                {"label": key, "pattern": sub_pattern1, "id": idnmb},
                {"label": key, "pattern": sub_pattern2, "id": idnmb},
                {"label": key, "pattern": sub_pattern3, "id": idnmb},
                {"label": key, "pattern": sub_pattern4, "id": idnmb},
                {"label": key, "pattern": sub_pattern5, "id": idnmb},
                {"label": key, "pattern": sub_pattern6, "id": idnmb}
            ]
        elif " " in entity:
            sub_pattern1 = [{"LOWER": entity[:entity.find(" ")].lower()}, {"LOWER": entity[entity.find(" ") + 1:].lower()}]
            sub_pattern2 = [{"LOWER": entity[:entity.find(" ")].lower()}, {"TEXT": "-"}, {"LOWER": entity[entity.find(" ") + 1:].lower()}]
            sub_pattern3 = [{"LOWER": "-" + entity[:entity.find(" ")].lower()}, {"LOWER": entity[entity.find(" ") + 1:].lower()}]
            sub_pattern4 = [{"LOWER": "-" + entity[:entity.find(" ")].lower()}, {"TEXT": "-"}, {"LOWER": entity[entity.find(" ") + 1:].lower()}]
            pattern = [
                sub_pattern1,
                sub_pattern2,
                sub_pattern3,
                sub_pattern4   
            ]
            patterns = [
                {"label": key, "pattern": sub_pattern1, "id": idnmb},
                {"label": key, "pattern": sub_pattern2, "id": idnmb},
                {"label": key, "pattern": sub_pattern3, "id": idnmb},
                {"label": key, "pattern": sub_pattern4, "id": idnmb}
            ]
        elif "+" in entity:
            sub_pattern1 = [{"LOWER": entity.lower()}]
            sub_pattern2 = [{"LOWER": "-" + entity.lower()}]
            sub_pattern3 = [{"LOWER": {"REGEX": entity.replace("+", "\+").lower() + "/"}}]
            sub_pattern4 = [{"LOWER": {"REGEX": "/" + entity.replace("+", "\+").lower()}}]
            pattern = [
                sub_pattern1,
                sub_pattern2,
                sub_pattern3,
                sub_pattern4
            ]
            patterns = [
                {"label": key, "pattern": sub_pattern1, "id": idnmb},
                {"label": key, "pattern": sub_pattern2, "id": idnmb},
                {"label": key, "pattern": sub_pattern3, "id": idnmb},
                {"label": key, "pattern": sub_pattern4, "id": idnmb}
            ]
        elif "/" in entity:
            sub_pattern1 = [{"LOWER": {"REGEX": entity[:entity.find("#")].lower() + "#/"}}]
            sub_pattern2 = [{"LOWER": {"REGEX": "/" + entity[:entity.find("#")].lower()}}, {"LOWER": "#"}]
            pattern = [
                sub_pattern1,
                sub_pattern2
             ]
            patterns = [
                {"label": key, "pattern": sub_pattern1, "id": idnmb},
                {"label": key, "pattern": sub_pattern2, "id": idnmb}
            ]
        elif "#" in entity:
            sub_pattern1 = [{"LOWER": entity[:entity.find("#")].lower()}, {"LOWER": "#"}]
            sub_pattern2 = [{"LOWER": "-" + entity[:entity.find("#")].lower()}, {"LOWER": "#"}]
            pattern = [
                sub_pattern1,
                sub_pattern2
            ]
            patterns = [
                {"label": key, "pattern": sub_pattern1, "id": idnmb},
                {"label": key, "pattern": sub_pattern2, "id": idnmb}
            ]
        elif len(entity) == 1:
            sub_pattern1 = [{"LOWER": entity.lower()}]
            sub_pattern2 = [{"LOWER": "-" + entity.lower()}]
            sub_pattern3 = [{"LOWER": entity.lower() + "."}]
            sub_pattern4 = [{"LOWER": {"REGEX": "/" + entity.lower()}}]
            sub_pattern5 = [{"LOWER": {"REGEX": entity.lower() + "/"}}]
            pattern = [
                sub_pattern1,
                sub_pattern2,
                sub_pattern3,
                sub_pattern4,
                sub_pattern5
            ]
            patterns = [
                {"label": key, "pattern": sub_pattern1, "id": idnmb},
                {"label": key, "pattern": sub_pattern2, "id": idnmb},
                {"label": key, "pattern": sub_pattern3, "id": idnmb},
                {"label": key, "pattern": sub_pattern4, "id": idnmb},
                {"label": key, "pattern": sub_pattern5, "id": idnmb}
            ]
        elif entity[0] == "A" or entity[0] == "E" or entity[0] == "H" or entity[0] == "I" or entity[0] == "O" or entity[0] == "U":
            sub_pattern1 = [{"LOWER": entity.lower()}]
            sub_pattern2 = [{"LOWER": "-" + entity.lower()}]
            sub_pattern3 = [{"LOWER": {"REGEX": "/" + entity.lower()}}]
            sub_pattern4 = [{"LOWER": {"REGEX": entity.lower() + "/"}}]
            sub_pattern5 = [{"LOWER": "d'" + entity.lower()}]
            sub_pattern6 = [{"LOWER": "-" + "d'" + entity.lower()}]
            sub_pattern7 = [{"LOWER": {"REGEX": "/" + "d'" + entity.lower()}}]
            sub_pattern8 = [{"LOWER": {"REGEX": "d'" + entity.lower() + "/"}}]
            sub_pattern9 = [{"LOWER": "l'" + entity.lower()}]
            sub_pattern10 = [{"LOWER": "-" + "l'" + entity.lower()}]
            sub_pattern11 = [{"LOWER": {"REGEX": "/" + "l'" + entity.lower()}}]
            sub_pattern12 = [{"LOWER": {"REGEX": "l'" + entity.lower() + "/"}}]
            sub_pattern13 = [{"LOWER": "d’" + entity.lower()}]
            sub_pattern14 = [{"LOWER": "-" + "d’" + entity.lower()}]
            sub_pattern15 = [{"LOWER": {"REGEX": "/" + "d’" + entity.lower()}}]
            sub_pattern16 = [{"LOWER": {"REGEX": "d’" + entity.lower() + "/"}}]
            sub_pattern17 = [{"LOWER": "l’" + entity.lower()}]
            sub_pattern18 = [{"LOWER": "-" + "l’" + entity.lower()}]
            sub_pattern19 = [{"LOWER": {"REGEX": "/" + "l’" + entity.lower()}}]
            sub_pattern20 = [{"LOWER": {"REGEX": "l’" + entity.lower() + "/"}}]
            pattern = [
                sub_pattern1,
                sub_pattern2,
                sub_pattern3,
                sub_pattern4,
                sub_pattern5,
                sub_pattern6,
                sub_pattern7,
                sub_pattern8,
                sub_pattern9,
                sub_pattern10,
                sub_pattern11,
                sub_pattern12,
                sub_pattern13,
                sub_pattern14,
                sub_pattern15,
                sub_pattern16,
                sub_pattern17,
                sub_pattern18,
                sub_pattern19,
                sub_pattern20
            ]
            patterns = [
                {"label": key, "pattern": sub_pattern1, "id": idnmb},
                {"label": key, "pattern": sub_pattern2, "id": idnmb},
                {"label": key, "pattern": sub_pattern3, "id": idnmb},
                {"label": key, "pattern": sub_pattern4, "id": idnmb},
                {"label": key, "pattern": sub_pattern5, "id": idnmb},
                {"label": key, "pattern": sub_pattern6, "id": idnmb},
                {"label": key, "pattern": sub_pattern7, "id": idnmb},
                {"label": key, "pattern": sub_pattern8, "id": idnmb},
                {"label": key, "pattern": sub_pattern9, "id": idnmb},
                {"label": key, "pattern": sub_pattern10, "id": idnmb},
                {"label": key, "pattern": sub_pattern11, "id": idnmb},
                {"label": key, "pattern": sub_pattern12, "id": idnmb},
                {"label": key, "pattern": sub_pattern13, "id": idnmb},
                {"label": key, "pattern": sub_pattern14, "id": idnmb},
                {"label": key, "pattern": sub_pattern15, "id": idnmb},
                {"label": key, "pattern": sub_pattern16, "id": idnmb},
                {"label": key, "pattern": sub_pattern17, "id": idnmb},
                {"label": key, "pattern": sub_pattern18, "id": idnmb},
                {"label": key, "pattern": sub_pattern19, "id": idnmb},
                {"label": key, "pattern": sub_pattern20, "id": idnmb}
            ]
        elif "(s)" in entity:
            sub_pattern1 = [{"LOWER": entity[:entity.find("(")].lower()}]
            sub_pattern2 = [{"LOWER": "-" + entity[:entity.find("(")].lower()}]
            sub_pattern3 = [{"LOWER": {"REGEX": "/" + entity[:entity.find("(")].lower()}}]
            sub_pattern4 = [{"LOWER": {"REGEX": entity[:entity.find("(")].lower() + "/"}}]
            sub_pattern5 = [{"LOWER": entity[:entity.find("(")].lower() + "s"}]
            sub_pattern6 = [{"LOWER": "-" + entity[:entity.find("(")].lower() + "s"}]
            sub_pattern7 = [{"LOWER": {"REGEX": "/" + entity[:entity.find("(")].lower() + "s"}}]
            sub_pattern8 = [{"LOWER": {"REGEX": entity[:entity.find("(")].lower() + "s" + "/"}}]
            pattern = [
                sub_pattern1,
                sub_pattern2,
                sub_pattern3,
                sub_pattern4,
                sub_pattern5,
                sub_pattern6,
                sub_pattern7,
                sub_pattern8
            ]
            patterns = [
                {"label": key, "pattern": sub_pattern1, "id": idnmb},
                {"label": key, "pattern": sub_pattern2, "id": idnmb},
                {"label": key, "pattern": sub_pattern3, "id": idnmb},
                {"label": key, "pattern": sub_pattern4, "id": idnmb},
                {"label": key, "pattern": sub_pattern5, "id": idnmb},
                {"label": key, "pattern": sub_pattern6, "id": idnmb},
                {"label": key, "pattern": sub_pattern7, "id": idnmb},
                {"label": key, "pattern": sub_pattern8, "id": idnmb}
            ]
        else:
            sub_pattern1 = [{"LOWER": entity.lower()}]
            sub_pattern2 = [{"LOWER": "-" + entity.lower()}]
            sub_pattern3 = [{"LOWER": {"REGEX": "/" + entity.lower()}}]
            sub_pattern4 = [{"LOWER": {"REGEX": entity.lower() + "/"}}]
            pattern = [
                sub_pattern1,
                sub_pattern2,
                sub_pattern3,
                sub_pattern4
            ]
            patterns = [
                {"label": key, "pattern": sub_pattern1, "id": idnmb},
                {"label": key, "pattern": sub_pattern2, "id": idnmb},
                {"label": key, "pattern": sub_pattern3, "id": idnmb},
                {"label": key, "pattern": sub_pattern4, "id": idnmb}
            ]
        matcher.add(idnmb, pattern)
        ruler.add_patterns(patterns)

In [10]:
jobs = pd.read_csv('SimplyHired_CAN_JD_AUG_2022.csv')

In [11]:
jobs

,Title,Company,Location,Qualifications,JobDescription
0,AAVAA - Machine Learning and Signal Processing...,AAVAA,"Montréal, QC",TensorFlow\nC++\nMATLAB\nFirmware\nC,Full job description\nMachine Learning and Sig...
1,Data Engineer,GoTo,"Montréal, QC",SQL\nSpark\nApache Hive\nCommunication skills\...,Full job description\nJob Description\n\nOur C...
2,Senior Cloud Infrastructure Engineer (VMware/HCI),Scotiabank,"Toronto, ON",REST\nVMWare\nDisaster recovery\nWindows\nMicr...,Full job description\nRequisition ID: 154917\n...
3,Director of Engineering,RBC Financial Group,"Toronto, ON",Communication skills\nAnalytics\nMachine learning,Full job description\nCome Work with Us!\n\nAt...
4,Data scientist and machine learning engineer w...,PredictNow.ai,"Toronto, ON",Machine learning,Full job description\nWe are looking to comple...
...,...,...,...,...,...
572,Software Engineer (Backend),EarthDaily Analytics,"Vancouver, BC",Azure\nNumPy\nSciPy\nC++\nDevOps,Full job description\nOUR VISION\n\nAt EarthDa...
573,"Software Developer, 5G RAN Transport - Ottawa ...",Dell Technologies,"Ottawa, ON",Encryption,Full job description\n5G Transport Intermediat...
574,5G RAN Transport Senior Developer - Ottawa or ...,Dell Technologies,"Ottawa, ON",Encryption,Full job description\n5G RAN Transport Senior ...
575,Senior Linux Kernel Software Developer,"Advanced Micro Devices, Inc.","Markham, ON",Perl\nOS Kernels\nShell Scripting\nC++\nCommun...,Full job description\nWhat you do at AMD chang...


In [12]:
num = 0
for description in jobs['JobDescription']:
    print(str(num) + ' ' + description + '\n')
    num += 1

0 Full job description
Machine Learning and Signal Processing Specialist

We are looking for a full-time Machine Learning and Signal Processing Specialist as early as September 2022. With AAVAA, you will be part of a multidisciplinary team building a unique listening device that allows everyone to hear what they want by processing their surrounding sounds based on input from their brain and biosignals. Our goal is to give people control of their auditory environment, enhance the sounds they want to hear, and suppress distracting noise. In addition to solving one of the biggest challenges of human health, you will get:

A generous salary, benefits, and vacation program
A safe, healthy, and inclusive work environment
Healthy snacks, frequent celebrations, endless tea and latte, social outings, and more!

We are looking for problem solver engineers and scientists who have a collaborative and meticulous spirit. Applicants of all levels are welcome, including Senior Leads, Engineers, and Po

In [13]:
def get_nodeid(tech):
    index_num = dictionary[dictionary['name'] == tech].index.values
    index_number = index_num[0]
    return str(dictionary.at[index_number, 'nodeid'])

In [14]:
for numb in jobs.index:
    doc = nlp(jobs.at[numb,'JobDescription'])
    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        temp.append(string_id)
    for word in temp:
        cnt[word] += 1
    if cnt[get_nodeid('Android')]:
        cnt[get_nodeid('Android')] = cnt[get_nodeid('Android')] - cnt[get_nodeid('Android Studio')]
    if cnt[get_nodeid('Argo')]:
        cnt[get_nodeid('Argo')] = cnt[get_nodeid('Argo')] - cnt[get_nodeid('Argo CD')]
    if cnt[get_nodeid('Azure')]:
        cnt[get_nodeid('Azure')] = cnt[get_nodeid('Azure')] - cnt[get_nodeid('Azure Data Lake')] - cnt[get_nodeid('Azure DevOps')] - cnt[get_nodeid('Azure Machine Learning')] - cnt[get_nodeid('Azure SQL')]
    if cnt[get_nodeid('C')]:
        cnt[get_nodeid('C')] = cnt[get_nodeid('C')] - cnt[get_nodeid('Objective C')]
    if cnt[get_nodeid('R')]:
        cnt[get_nodeid('R')] = cnt[get_nodeid('R')] - cnt[get_nodeid('R Shiny')]
    if cnt[get_nodeid('React')]:
        cnt[get_nodeid('React')] = cnt[get_nodeid('React')] - cnt[get_nodeid('React Native')]
    if cnt[get_nodeid('Ruby')]:
        cnt[get_nodeid('Ruby')] = cnt[get_nodeid('Ruby')] - cnt[get_nodeid('Ruby on Rails')]
    if cnt[get_nodeid('SQL')]:
        cnt[get_nodeid('SQL')] = cnt[get_nodeid('SQL')] - cnt[get_nodeid('Azure SQL')] - cnt[get_nodeid('Microsoft SQL Server')]
    if cnt[get_nodeid('Visual Studio')]:
        cnt[get_nodeid('Visual Studio')] = cnt[get_nodeid('Visual Studio')] - cnt[get_nodeid('Visual Studio Code')]
    count = dict(cnt)
    tech_count.append(count)
    cnt.clear()
    temp.clear()
    print(numb)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [15]:
print(tech_count)

[{'930001': 1, '930000': 1, '930002': 1, '930120': 1, '930115': 1, '930103': 1, '930102': 1, '930101': 1, '930701': 1}, {'930200': 2, '930010': 3, '930353': 2, '930330': 2, '100169': 1, '93091': 1, '93025': 1, '930333': 1, '930120': 1, '930114': 1, '930106': 1, '930300': 1}, {'93064': 5, '100536': 2, '93100': 1, '93051': 1, '930500': 1, '930561': 1, '930120': 1, '930105': 1}, {'93025': 1, '930103': 1}, {}, {'930120': 1, '930106': 1, '930101': 1, '930114': 1, '100209': 1, '930020': 1, '930000': 1, '930002': 1, '930001': 1, '930010': 2, '930021': 1, '930333': 1, '930341': 1, '93061': 1, '93060': 1, '930331': 1, '100290': 1}, {'930001': 2, '930103': 1, '930120': 1}, {}, {'930120': 1, '930000': 1, '930001': 1, '930200': 1, '930010': 1, '93095': 1}, {'930105': 1}, {'930105': 1, '930120': 1, '930114': 1, '930010': 2, '930106': 1, '930000': 1, '930001': 1, '930020': 1, '930300': 1, '930333': 1, '930353': 1, '930119': 1, '100032': 1, '930441': 1, '930440': 1, '100102': 1, '100481': 1, '930700'

In [16]:
jobs['TechCount'] = tech_count

In [17]:
jobs

,Title,Company,Location,Qualifications,JobDescription,TechCount
0,AAVAA - Machine Learning and Signal Processing...,AAVAA,"Montréal, QC",TensorFlow\nC++\nMATLAB\nFirmware\nC,Full job description\nMachine Learning and Sig...,"{'930001': 1, '930000': 1, '930002': 1, '93012..."
1,Data Engineer,GoTo,"Montréal, QC",SQL\nSpark\nApache Hive\nCommunication skills\...,Full job description\nJob Description\n\nOur C...,"{'930200': 2, '930010': 3, '930353': 2, '93033..."
2,Senior Cloud Infrastructure Engineer (VMware/HCI),Scotiabank,"Toronto, ON",REST\nVMWare\nDisaster recovery\nWindows\nMicr...,Full job description\nRequisition ID: 154917\n...,"{'93064': 5, '100536': 2, '93100': 1, '93051':..."
3,Director of Engineering,RBC Financial Group,"Toronto, ON",Communication skills\nAnalytics\nMachine learning,Full job description\nCome Work with Us!\n\nAt...,"{'93025': 1, '930103': 1}"
4,Data scientist and machine learning engineer w...,PredictNow.ai,"Toronto, ON",Machine learning,Full job description\nWe are looking to comple...,{}
...,...,...,...,...,...,...
572,Software Engineer (Backend),EarthDaily Analytics,"Vancouver, BC",Azure\nNumPy\nSciPy\nC++\nDevOps,Full job description\nOUR VISION\n\nAt EarthDa...,"{'930200': 1, '930230': 1, '930220': 1, '93012..."
573,"Software Developer, 5G RAN Transport - Ottawa ...",Dell Technologies,"Ottawa, ON",Encryption,Full job description\n5G Transport Intermediat...,{}
574,5G RAN Transport Senior Developer - Ottawa or ...,Dell Technologies,"Ottawa, ON",Encryption,Full job description\n5G RAN Transport Senior ...,{}
575,Senior Linux Kernel Software Developer,"Advanced Micro Devices, Inc.","Markham, ON",Perl\nOS Kernels\nShell Scripting\nC++\nCommun...,Full job description\nWhat you do at AMD chang...,"{'930105': 1, '930500': 4, '930103': 1, '93010..."


In [18]:
jobs.to_csv('TechCount_SimplyHired_CAN_JD_AUG_2022.csv', index = False)